# 单例模式 (Singleton Pattern) 实战演示

## 概述

单例模式是一种创建型设计模式，它确保一个类只有一个实例，并提供全局访问点。

本Notebook将通过可执行的代码演示：
- 四种不同的单例实现方式
- 线程安全性测试
- 性能对比分析
- 实际应用场景

## 核心特征

- **唯一实例**：确保类只有一个实例
- **全局访问**：提供全局访问点
- **延迟初始化**：按需创建实例（部分实现）
- **线程安全**：多线程环境下的安全保障

## 1️⃣ 简单单例（非线程安全）

最基础的实现，适用于单线程环境：

In [ ]:
public sealed class SimpleSingleton
{
    private SimpleSingleton() 
    { 
        Console.WriteLine("🏗️ SimpleSingleton 实例被创建");
    }
    
    private static SimpleSingleton? _instance;

    public static SimpleSingleton GetInstance() 
    {
        if (_instance == null)
        {
            _instance = new SimpleSingleton();
        }
        return _instance;
    }
    
    public void DoWork()
    {
        Console.WriteLine("SimpleSingleton 正在工作...");
    }
}

// 测试简单单例
Console.WriteLine("=== 简单单例测试 ===");
var simple1 = SimpleSingleton.GetInstance();
var simple2 = SimpleSingleton.GetInstance();

Console.WriteLine($"实例1: {simple1.GetHashCode()}");
Console.WriteLine($"实例2: {simple2.GetHashCode()}");
Console.WriteLine($"是同一个实例吗？ {object.ReferenceEquals(simple1, simple2)}");

## 2️⃣ 线程安全单例（双重检查锁）

使用锁机制确保线程安全：

In [ ]:
public sealed class ThreadSafeSingleton
{
    private ThreadSafeSingleton() 
    { 
        Console.WriteLine("🏗️ ThreadSafeSingleton 实例被创建");
    }
    
    private static ThreadSafeSingleton? _instance;
    private static readonly object _lock = new object();

    public static ThreadSafeSingleton GetInstance()
    {
        // 第一次检查：避免已初始化后的锁开销
        if (_instance == null)
        {
            lock (_lock)
            {
                // 第二次检查：防止重复初始化
                if (_instance == null)
                {
                    _instance = new ThreadSafeSingleton();
                }
            }
        }
        return _instance;
    }
    
    public void DoWork()
    {
        Console.WriteLine("ThreadSafeSingleton 正在工作...");
    }
}

// 测试线程安全单例
Console.WriteLine();
Console.WriteLine("=== 线程安全单例测试 ===");
var threadSafe1 = ThreadSafeSingleton.GetInstance();
var threadSafe2 = ThreadSafeSingleton.GetInstance();

Console.WriteLine($"实例1: {threadSafe1.GetHashCode()}");
Console.WriteLine($"实例2: {threadSafe2.GetHashCode()}");
Console.WriteLine($"是同一个实例吗？ {object.ReferenceEquals(threadSafe1, threadSafe2)}");

## 3️⃣ 静态单例（饿汉式）

在类加载时就创建实例：

In [ ]:
public sealed class StaticSingleton
{
    static StaticSingleton() 
    {
        Console.WriteLine("🏗️ StaticSingleton 静态构造函数被调用");
    }

    private StaticSingleton() 
    { 
        Console.WriteLine("🏗️ StaticSingleton 实例被创建");
    }
    
    private static readonly StaticSingleton _instance = new StaticSingleton();

    public static StaticSingleton GetInstance() => _instance;
    
    public void DoWork()
    {
        Console.WriteLine("StaticSingleton 正在工作...");
    }
}

// 测试静态单例
Console.WriteLine();
Console.WriteLine("=== 静态单例测试 ===");
var static1 = StaticSingleton.GetInstance();
var static2 = StaticSingleton.GetInstance();

Console.WriteLine($"实例1: {static1.GetHashCode()}");
Console.WriteLine($"实例2: {static2.GetHashCode()}");
Console.WriteLine($"是同一个实例吗？ {object.ReferenceEquals(static1, static2)}");

## 4️⃣ Lazy&lt;T&gt; 单例（推荐）

使用 .NET 内置的 Lazy&lt;T&gt; 类：

In [ ]:
public sealed class LazySingleton
{
    private LazySingleton() 
    { 
        Console.WriteLine("🏗️ LazySingleton 实例被创建");
    }
    
    private static readonly Lazy<LazySingleton> _instance = new Lazy<LazySingleton>(() => new LazySingleton());

    public static LazySingleton GetInstance() => _instance.Value;
    
    public void DoWork()
    {
        Console.WriteLine("LazySingleton 正在工作...");
    }
}

// 测试 Lazy 单例
Console.WriteLine();
Console.WriteLine("=== Lazy 单例测试 ===");
var lazy1 = LazySingleton.GetInstance();
var lazy2 = LazySingleton.GetInstance();

Console.WriteLine($"实例1: {lazy1.GetHashCode()}");
Console.WriteLine($"实例2: {lazy2.GetHashCode()}");
Console.WriteLine($"是同一个实例吗？ {object.ReferenceEquals(lazy1, lazy2)}");

## 🧪 多线程安全测试

让我们测试一下各种实现在多线程环境下的行为：

In [ ]:
using System.Collections.Concurrent;

// 多线程测试函数
async Task TestSingletonThreadSafety<T>(string name, Func<T> getInstance) where T : class
{
    const int taskCount = 100;
    var instances = new ConcurrentBag<T>();
    
    var tasks = Enumerable.Range(0, taskCount)
        .Select(_ => Task.Run(() => instances.Add(getInstance())))
        .ToArray();
    
    await Task.WhenAll(tasks);
    
    var uniqueInstances = instances.Distinct().Count();
    Console.WriteLine($"{name}: 创建了 {uniqueInstances} 个不同的实例 (应该为1)");
    
    if (uniqueInstances == 1)
    {
        Console.WriteLine("✅ 线程安全");
    }
    else
    {
        Console.WriteLine("❌ 线程不安全");
    }
}

Console.WriteLine("🔄 开始多线程安全测试...");
Console.WriteLine();

In [ ]:
// 重新定义一个测试用的简单单例（可能有线程安全问题）
public sealed class TestSimpleSingleton
{
    private TestSimpleSingleton() { }
    private static TestSimpleSingleton? _instance;
    
    public static TestSimpleSingleton GetInstance()
    {
        if (_instance == null)
        {
            Thread.Sleep(1); // 增加竞态条件的可能性
            _instance = new TestSimpleSingleton();
        }
        return _instance;
    }
}

// 执行多线程测试
await TestSingletonThreadSafety("简单单例", () => TestSimpleSingleton.GetInstance());
await TestSingletonThreadSafety("线程安全单例", () => ThreadSafeSingleton.GetInstance());
await TestSingletonThreadSafety("静态单例", () => StaticSingleton.GetInstance());
await TestSingletonThreadSafety("Lazy单例", () => LazySingleton.GetInstance());

## 🎯 实际应用示例

让我们看看单例模式在实际开发中的应用：

In [ ]:
// 配置管理器示例
public sealed class ConfigurationManager
{
    private static readonly Lazy<ConfigurationManager> _instance = 
        new Lazy<ConfigurationManager>(() => new ConfigurationManager());
    
    private readonly Dictionary<string, string> _settings;
    
    private ConfigurationManager()
    {
        Console.WriteLine("🔧 加载配置文件...");
        _settings = new Dictionary<string, string>
        {
            ["DatabaseConnection"] = "Server=localhost;Database=MyApp;",
            ["LogLevel"] = "Information",
            ["MaxRetries"] = "3"
        };
    }
    
    public static ConfigurationManager Instance => _instance.Value;
    
    public string GetSetting(string key) => 
        _settings.TryGetValue(key, out var value) ? value : "未找到配置";
        
    public void SetSetting(string key, string value) => 
        _settings[key] = value;
}

// 日志记录器示例
public sealed class Logger
{
    private static readonly Lazy<Logger> _instance = new Lazy<Logger>(() => new Logger());
    
    private Logger() 
    {
        Console.WriteLine("📝 初始化日志记录器...");
    }
    
    public static Logger Instance => _instance.Value;
    
    public void LogInfo(string message) => 
        Console.WriteLine($"[INFO {DateTime.Now:HH:mm:ss}] {message}");
        
    public void LogError(string message) => 
        Console.WriteLine($"[ERROR {DateTime.Now:HH:mm:ss}] {message}");
}

Console.WriteLine("=== 实际应用演示 ===");

In [ ]:
// 使用配置管理器
var config = ConfigurationManager.Instance;
Console.WriteLine($"数据库连接: {config.GetSetting("DatabaseConnection")}");
Console.WriteLine($"日志级别: {config.GetSetting("LogLevel")}");

config.SetSetting("NewSetting", "新配置值");
Console.WriteLine($"新配置: {config.GetSetting("NewSetting")}");

Console.WriteLine();

// 使用日志记录器
var logger = Logger.Instance;
logger.LogInfo("应用程序启动");
logger.LogError("模拟错误信息");

Console.WriteLine();

// 验证单例特性
var config2 = ConfigurationManager.Instance;
var logger2 = Logger.Instance;

Console.WriteLine($"配置管理器是同一实例吗？ {object.ReferenceEquals(config, config2)}");
Console.WriteLine($"日志记录器是同一实例吗？ {object.ReferenceEquals(logger, logger2)}");

## 📊 性能对比

让我们比较不同单例实现的性能：

In [ ]:
using System.Diagnostics;

// 性能测试函数
void BenchmarkSingleton<T>(string name, Func<T> getInstance, int iterations = 1000000)
{
    var sw = Stopwatch.StartNew();
    
    for (int i = 0; i < iterations; i++)
    {
        var instance = getInstance();
    }
    
    sw.Stop();
    Console.WriteLine($"{name}: {iterations:N0} 次调用耗时 {sw.ElapsedMilliseconds} ms");
}

Console.WriteLine("⚡ 性能测试开始...");
Console.WriteLine("测试 1,000,000 次 GetInstance() 调用:");
Console.WriteLine();

// 注意：第一次调用会触发实例创建，后续调用只是返回已有实例
BenchmarkSingleton("线程安全单例", () => ThreadSafeSingleton.GetInstance());
BenchmarkSingleton("静态单例", () => StaticSingleton.GetInstance());
BenchmarkSingleton("Lazy单例", () => LazySingleton.GetInstance());

Console.WriteLine();
Console.WriteLine("📈 结论: 静态单例通常性能最好，Lazy单例略慢但更灵活，线程安全单例有锁开销");

## 📝 总结

### 🎯 单例模式的关键点

| 实现方式 | 线程安全 | 延迟初始化 | 性能 | 推荐场景 |
|---------|---------|-----------|------|---------|
| 简单单例 | ❌ | ✅ | 高 | 单线程应用 |
| 线程安全单例 | ✅ | ✅ | 中 | 需要精确控制 |
| 静态单例 | ✅ | ❌ | 高 | 启动时必需 |
| Lazy单例 | ✅ | ✅ | 高 | **推荐使用** |

### ✅ 优势
- **控制实例数量**：确保全局唯一
- **节省资源**：避免重复创建
- **全局访问**：提供统一接入点

### ⚠️ 注意事项
- **避免过度使用**：可能导致紧耦合
- **测试困难**：全局状态难以模拟
- **线程安全**：多线程环境需要特别注意

### 🔄 现代替代方案
- **依赖注入容器**：更好的可测试性和管理
- **服务定位器模式**：更灵活的服务管理